In [1]:
import os
import sys

# run locally without install asynfed package
root = os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd())))
sys.path.append(root)


In [4]:
# tensorflow 
from asynfed.client.frameworks.tensorflow.tensorflow_framework import TensorflowFramework
from resnet18 import Resnet18
from data_preprocessing import *


In [5]:
batch_size = 128
epoch = 5

In [6]:

# load data
default_training_dataset_path = "../../data/cifar_data/5_chunks/chunk_2.pickle"
default_testing_dataset_path = "../../data/cifar_data/test_set.pickle"


# train_ds, data_size = preprocess_dataset(training_dataset_path, training = True)
# test_ds, _ = preprocess_dataset(testing_dataset_path, training = False)
train_ds, data_size = preprocess_dataset(default_training_dataset_path, batch_size = 128, training = True)
test_ds, _ = preprocess_dataset(default_testing_dataset_path, training = False)


2023-07-02 10:24:02.033083: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [7]:

# define model
model = Resnet18(input_features = (32, 32, 3), output_features = 10, lr=1e-1, decay_steps=int(epoch * data_size / batch_size))
# define framework
tensorflow_framework = TensorflowFramework(model = model, epoch= 200, data_size= data_size, train_ds= train_ds, test_ds= test_ds, regularization='l2', delta_time= 10000, qod= 0.45)



In [8]:
# fit to be able to load model
for images, labels in tensorflow_framework.train_ds:
    train_acc, train_loss= tensorflow_framework.fit(images, labels)
    break


2023-07-02 10:24:07.573078: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [4166,10]
	 [[{{node Placeholder/_1}}]]
2023-07-02 10:24:07.573316: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [4166,10]
	 [[{{node Placeholder/_1}}]]


In [ ]:
# training_process/cifar_dataset/server/weights/global_weights/d2771e5c-4cd7-470e-a376-aa67686a0ebc_v147.pkl

In [21]:
n = 140
for i in range(20):
    version = n + i
    file_name = f"./weights/global_weights/d2771e5c-4cd7-470e-a376-aa67686a0ebc_v{version}.pkl"

    with open(file_name, "rb") as f:
        import pickle
        weights = pickle.load(f)

    tensorflow_framework.set_weights(weights)

    tensorflow_framework.model.test_loss.reset_states()
    tensorflow_framework.model.test_performance.reset_states()

    for test_images, test_labels in tensorflow_framework.test_ds:
        test_acc, test_loss = tensorflow_framework.evaluate(test_images, test_labels)

    print("Global Version {}: Test Acc {:.2f}  Test Loss {}".format(version, test_acc  * 100, test_loss) )
    

Global Version 140: Test Acc 85.70  Test Loss 1.8468708992004395
Global Version 141: Test Acc 72.21  Test Loss 5.635708332061768
Global Version 142: Test Acc 85.19  Test Loss 1.9556220769882202
Global Version 143: Test Acc 67.11  Test Loss 8.7385892868042
Global Version 144: Test Acc 87.68  Test Loss 1.433874487876892
Global Version 145: Test Acc 78.78  Test Loss 4.103888988494873
Global Version 146: Test Acc 70.81  Test Loss 7.685883045196533
Global Version 147: Test Acc 59.41  Test Loss 15.502869606018066
Global Version 148: Test Acc 90.15  Test Loss 1.0208758115768433
Global Version 149: Test Acc 90.08  Test Loss 1.1057039499282837
Global Version 150: Test Acc 88.68  Test Loss 1.44014310836792
Global Version 151: Test Acc 87.45  Test Loss 1.8236335515975952
Global Version 152: Test Acc 90.31  Test Loss 0.9900872111320496
Global Version 153: Test Acc 90.17  Test Loss 1.0291190147399902
Global Version 154: Test Acc 88.96  Test Loss 1.455822467803955
Global Version 155: Test Acc 87.76 